# 1.데이터셋 수집

## 공개데이터셋 가져오기

### iWildCam Dataset 특정 데이터 가져오기

In [2]:
import json

# 경로 설정 (데이터셋의 주석 파일 위치)
annotations_path = "E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\data\iwildcam2022_train_annotations.json"

# 주석 파일 열기
with open(annotations_path, "r") as f:
    annotations = json.load(f)


In [ ]:
print(annotations.keys())

dict_keys(['images', 'annotations', 'categories'])


### 클래스별 폴더에 이미지 분류

In [ ]:
import json

# 경로 설정 (데이터셋의 주석 파일 위치)
annotations_path = "E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\data\iwildcam2022_train_annotations.json"

# 주석 파일 열기
with open(annotations_path, "r") as f:
    annotations = json.load(f)

# 클래스 정보 추출
categories = annotations["categories"]

# 클래스 ID와 이름 출력
print("클래스 종류:")
for category in categories:
    print(f"ID: {category['id']}, Name: {category['name']}")

클래스 종류:
ID: 0, Name: empty
ID: 2, Name: tayassu pecari
ID: 3, Name: dasyprocta punctata
ID: 4, Name: cuniculus paca
ID: 6, Name: puma concolor
ID: 7, Name: tapirus terrestris
ID: 8, Name: pecari tajacu
ID: 9, Name: mazama americana
ID: 10, Name: leopardus pardalis
ID: 12, Name: geotrygon montana
ID: 13, Name: nasua nasua
ID: 14, Name: dasypus novemcinctus
ID: 15, Name: eira barbara
ID: 16, Name: didelphis marsupialis
ID: 20, Name: procyon cancrivorus
ID: 24, Name: panthera onca
ID: 25, Name: myrmecophaga tridactyla
ID: 26, Name: tinamus major
ID: 32, Name: sylvilagus brasiliensis
ID: 44, Name: puma yagouaroundi
ID: 50, Name: leopardus wiedii
ID: 62, Name: mazama gouazoubira
ID: 67, Name: philander opossum
ID: 70, Name: capra aegagrus
ID: 71, Name: bos taurus
ID: 72, Name: ovis aries
ID: 73, Name: canis lupus
ID: 74, Name: lepus saxatilis
ID: 77, Name: turtur calcospilos
ID: 78, Name: papio anubis
ID: 80, Name: genetta genetta
ID: 83, Name: tragelaphus scriptus
ID: 86, Name: equus afric

In [ ]:
import json
import os
import shutil

# JSON 파일 경로
annotations_path = "E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\iwildcam2022_train_annotations.json"  # JSON 파일 경로

# 이미지 파일 경로
image_dir = "E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\iwildcam Dataset"  # 이미지 폴더 경로

# 분류된 이미지를 저장할 폴더
output_dir = "path_to_output_folder"
os.makedirs(output_dir, exist_ok=True)

# JSON 파일 읽기
with open(annotations_path, "r") as f:
    annotations = json.load(f)

# 클래스별로 이미지 분류
for annotation in annotations["annotations"]:
    category_id = annotation["category_id"]  # 클래스 ID
    image_id = annotation["image_id"]       # 이미지 ID
    image_file = f"{image_id}.jpg"          # 이미지 파일 이름 (확장자 확인 필요)

    # 클래스별 폴더 생성
    class_folder = os.path.join(output_dir, f"class_{category_id}")
    os.makedirs(class_folder, exist_ok=True)

    # 이미지 복사
    source_path = os.path.join(image_dir, image_file)
    dest_path = os.path.join(class_folder, image_file)

    if os.path.exists(source_path):  # 이미지가 존재할 경우만 복사
        shutil.move(source_path, dest_path)

print("이미지 분류 작업 완료!")


In [12]:
import json

# 경로 설정 (데이터셋의 주석 파일 위치)
bbox_labels_path = r"E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\bbox_labels_600_hierarchy.json"

# 주석 파일 열기
with open(bbox_labels_path, "r") as f:
    bbox_labels= json.load(f)

## 크롤링

### Google 을 통해 크롤링 하기

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import base64
import os

# Selenium WebDriver 설정
driver = webdriver.Chrome()
driver.get("https://www.google.com/imghp")

# 검색어 입력 및 실행
search_box = driver.find_element(By.NAME, "q")
search_box.send_keys("weasel")
search_box.send_keys(Keys.RETURN)

# 스크롤 및 추가 결과 로드
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        try:
            driver.find_element(By.CSS_SELECTOR, ".mye4qd").click()  # "결과 더보기" 버튼 클릭
        except:
            break
    last_height = new_height

# 이미지 태그 가져오기
image_elements = driver.find_elements(By.CSS_SELECTOR, "g-img img")

# 저장 폴더 생성
output_dir = "E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\data\weasel"
os.makedirs(output_dir, exist_ok=True)

# 이미지 다운로드
for i, img_element in enumerate(image_elements[50:100]):  # 최대 50개 다운로드
    try:
        img_url = img_element.get_attribute("src")
        if img_url.startswith("http"):  # HTTP/HTTPS URL 처리
            response = requests.get(img_url)
            with open(f"{output_dir}/image_{i + 1}.jpg", "wb") as f:
                f.write(response.content)
            print(f"이미지 저장 완료: image_{i + 1}.jpg")
        elif img_url.startswith("data:image"):  # Base64 데이터 처리
            header, base64_data = img_url.split(",", 1)
            image_data = base64.b64decode(base64_data)
            with open(f"{output_dir}/image_{i + 1}.jpg", "wb") as f:
                f.write(image_data)
            print(f"Base64 이미지 저장 완료: image_{i + 1}.jpg")
        else:
            print(f"이미지 URL 무효: {img_url}")
    except Exception as e:
        print(f"이미지 저장 실패: {img_url}")
        print(f"에러 메시지: {e}")

# 브라우저 종료
driver.quit()
print("이미지 다운로드 완료!")


이미지 저장 완료: image_1.jpg
Base64 이미지 저장 완료: image_2.jpg
이미지 저장 완료: image_3.jpg
Base64 이미지 저장 완료: image_4.jpg
이미지 저장 완료: image_5.jpg
Base64 이미지 저장 완료: image_6.jpg
Base64 이미지 저장 완료: image_7.jpg
Base64 이미지 저장 완료: image_8.jpg
Base64 이미지 저장 완료: image_9.jpg
Base64 이미지 저장 완료: image_10.jpg
Base64 이미지 저장 완료: image_11.jpg
Base64 이미지 저장 완료: image_12.jpg
Base64 이미지 저장 완료: image_13.jpg
Base64 이미지 저장 완료: image_14.jpg
Base64 이미지 저장 완료: image_15.jpg
Base64 이미지 저장 완료: image_16.jpg
Base64 이미지 저장 완료: image_17.jpg
Base64 이미지 저장 완료: image_18.jpg
Base64 이미지 저장 완료: image_19.jpg
Base64 이미지 저장 완료: image_20.jpg
Base64 이미지 저장 완료: image_21.jpg
Base64 이미지 저장 완료: image_22.jpg
Base64 이미지 저장 완료: image_23.jpg
Base64 이미지 저장 완료: image_24.jpg
Base64 이미지 저장 완료: image_25.jpg
Base64 이미지 저장 완료: image_26.jpg
Base64 이미지 저장 완료: image_27.jpg
Base64 이미지 저장 완료: image_28.jpg
Base64 이미지 저장 완료: image_29.jpg
Base64 이미지 저장 완료: image_30.jpg
Base64 이미지 저장 완료: image_31.jpg
Base64 이미지 저장 완료: image_32.jpg
Base64 이미지 저장 완료: image_33.jpg
Base64 

### Bing 을 통해 크롤링 하기

In [1]:
import os
# pip install bing-image-downloader

from bing_image_downloader import downloader  # bing_image_downloader 모듈에서 downloader를 가져옴
from pathlib import Path  # pathlib 모듈에서 Path를 가져옴

def download_images(keyword, num_images, file_name):
    output_directory = output_dir  # 이미지를 저장할 폴더 이름
    os.makedirs(output_directory, exist_ok=True)

    downloader.download(keyword, limit=num_images, output_dir=output_directory, adult_filter_off=True, force_replace=False, timeout=60)  # 이미지 다운로드

    root = Path().cwd() / output_directory  # 이미지 저장 폴더의 경로
    downloaded_files = list(root.glob("*.*"))  # 폴더 내 모든 파일 가져오기
    for i, file in enumerate(downloaded_files):  # 다운로드된 각 파일에 대해 반복
        extension = file.suffix  # 파일 확장자
        new_file_name = f"{file_name}{i+100}{extension}"  # 새로운 파일 이름
        new_file_path = root / new_file_name  # 새로운 파일 경로
        file.rename(new_file_path)  # 파일 이름 변경

# 사용 예시
output_dir = 'E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\data\족제비2'
keyword = "type of weasel"  # 이미지 검색 키워드
num_images = 70  # 다운로드할 이미지 수
file_name = "저장파일"  # 파일 이름

download_images(keyword, num_images, file_name)  # 이미지 다운로드

[%] Downloading Images to E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\data\족제비2\type of weasel


[!!]Indexing page: 1

[%] Indexed 95 Images on Page 1.


[%] Downloading Image #1 from https://i.ytimg.com/vi/czmzgf4bfVU/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://www.animalspot.net/wp-content/uploads/2022/01/Weasel.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://i.pinimg.com/736x/54/82/65/54826540fb0d990f231e200674f95a7a.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://upload.wikimedia.org/wikipedia/commons/0/08/Mountain_Weasel_(Mustela_altaica).jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://images.squarespace-cdn.com/content/v1/569ec99b841abaccb7c7e74c/1648079255072-PZCFCPMCMJYRWXAX5YPT/87049112.EWLfUP25.ltweasel101007IMG_5066copy.jpg
[Error]Invalid image, not saving https://images.squarespace-cdn.com/content/v1/569ec99b841abaccb7c7e74c/1648079255072-PZCFCPMCMJYRWXAX5YPT/87049112.EWLfU

### You tube 동영상 다운로드

In [31]:
pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [37]:
from pytube import YouTube

pytube.__main__.YouTube.__cache__.clear()


NameError: name 'pytube' is not defined

In [38]:
from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=dp3yllV8yX0",
    add_video_info=True,
)

results = loader.load()

ModuleNotFoundError: No module named 'langchain_community'

In [33]:
from pytube import YouTube

# YouTube URL 설정
url = "https://www.youtube.com/watch?v=dp3yllV8yX0"

# YouTube 객체 생성
yt = YouTube(url)

# 동영상 정보 출력
print("Title:", yt.title)
print("Views:", yt.views)
print("Length (seconds):", yt.length)

output_path = 'E:\Desktop\AI learning\Object_detection_segmentation\파수꾼만들기\data\video'
# 동영상 다운로드 (최고 화질)
stream = yt.streams.get_highest_resolution()
print("Downloading...")
stream.download(output_path=output_path)  # 저장 경로 지정
print("Download complete!")


PytubeError: Exception while accessing title of https://youtube.com/watch?v=dp3yllV8yX0. Please file a bug report at https://github.com/pytube/pytube

#### 여러 동영상 다운로드


In [ ]:
urls = [
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "https://www.youtube.com/watch?v=3JZ_D3ELwOQ"
]

for url in urls:
    yt = YouTube(url)
    stream = yt.streams.get_highest_resolution()
    print(f"Downloading: {yt.title}")
    stream.download(output_path="./videos")
    print("Download complete!")


# 모델 학습

해당 모델 학습은 Colab에서 진행

### Colab 환경에 data 가져오기

In [ ]:
from genericpath import exists
import os
import zipfile

zip_path = '/content/drive/MyDrive/AI_Nvidia/object_detection/make_object_detection.v1i.yolov8.zip'
extract_to_path = '/content/detection_data'

os.makedirs(extract_to_path, exist_ok=True)

with zipfile.ZipFile(zip_path,'r') as zip_ref:
  zip_ref.extractall(extract_to_path)

print(f"압축완료: {extract_to_path}")

### 필요 라이브러리 임포트

In [ ]:
import os                   # 시스템 경로 및 파일 관련 기능을 사용하기 위한 라이브러리
import json                 # JSON 데이터를 다루기 위한 라이브러리
import glob                 # 파일 패턴 매칭을 위한 라이브러리
import numpy as np          # 다차원 배열 및 수학 연산 처리를 위한 라이브러리
import pandas as pd         # 데이터프레임을 활용한 데이터 조작 및 분석을 위한 라이브러리
from ast import literal_eval  # 문자열을 코드로 변환하는 함수
from PIL import Image, ImageDraw  # 이미지 처리를 위한 라이브러리
import matplotlib.pyplot as plt  # 시각화를 위한 라이브러리
from tqdm import tqdm        # 반복문의 진행 상황을 표시하는 라이브러리
import random                # 랜덤 함수 사용을 위한 라이브러리
import yaml                  # yaml파일을 다루기 위한 라이브러리
from collections import OrderedDict # collections에서 제공하는 항목의 순서를 기억하는 딕셔너리
import shutil                # 파일의 이동과 복사를 위한 라이브러리

### 모델가져오기

In [ ]:
!pip3 install ultralytics

In [ ]:
import ultralytics
from ultralytics import YOLO
from IPython.display import Image as show_image
import glob
import random
import matplotlib.pyplot as plt
ultralytics.checks()

### 내 데이터 학습시키기

In [ ]:
project = "make_pasukkun" #학습 결과가 저장될 프로젝트 폴더
name = "after_train_result" # 학습결과를 저장할 실험(experiment)폴더 이름
dataset_path = "/content/detection_data/data.yaml" #yaml 파일 경로

In [ ]:
model.train(
    data=dataset_path, # 데이터셋 경로(yaml파일)
    epochs=30, # 최대 학습 epoch
    project=project, # project 폴더 이름
    name=name, # experiment 폴더 이름
    exist_ok=False, # 덮어쓰기 여부
    seed=42, # 재현성을 위한 시드 설정
    optimizer='AdamW', # optimizer는 AdamW로 지정
    patience=5, # 30epoch동안 loss가 개선되지 않으면 종료
    batch=32, # 훈련 배치 사이즈는 32
    imgsz=640 # 이미지 사이즈는 640
)

In [ ]:
show_image(f"{project}/{name}/results.png", width=800)

### 옵션을 바꿔서 재학습

In [ ]:
model.train(
    data=dataset_path, # 데이터셋 경로(yaml파일)
    epochs=60, # 최대 학습 epoch
    project=project, # project 폴더 이름
    name=name, # experiment 폴더 이름
    resume=False,            # 추가 학습 대신 새로운 학습 시작
    model='make_pasukkun/after_train_result/weights/best.pt', # 이전 학습된 모델 사용
    exist_ok=False, # 덮어쓰기 여부
    seed=42, # 재현성을 위한 시드 설정
    optimizer='AdamW', # optimizer는 AdamW로 지정
    patience=5, # 30epoch동안 loss가 개선되지 않으면 종료
    batch=64, # 훈련 배치 사이즈는 16
    imgsz=640 # 이미지 사이즈는 640
)